In [1]:
from utils import *

# Genre Interaction Network

How are different game genres interconnected through shared developers or publishers, and what are the implications for genre diversification and cross-genre game development?

In [2]:
df = pd.read_csv('steam.csv', usecols=['genres', 'developer', 'publisher'])

In [3]:
row_list = []

for idx, row in (
    df
    .dropna()
    .applymap(lambda x: x.split(';'))
    .explode('genres')
    .explode('developer')
    .explode('publisher')
).iterrows():
    if row.developer != row.publisher:
        row_list.extend([
            (idx, row.genres, row.developer),
            (idx, row.genres, row.publisher)
        ])
    elif row.developer == row.publisher:
        row_list.append((idx, row.genres, row.developer))

df_genre = (
    pd.DataFrame(
        row_list, columns=['release_date', 'genre', 'developer_publisher']
    )
    .set_index('release_date')
)

df_genre.head(10)

,genre,developer_publisher
release_date,,
0,Action,Valve
1,Action,Valve
2,Action,Valve
3,Action,Valve
4,Action,Gearbox Software
4,Action,Valve
5,Action,Valve
6,Action,Valve
7,Action,Valve


In [4]:
genres = set(df_genre['genre'])

shared_dev_pub = {}

for genre1, genre2 in itertools.combinations(genres, 2):
    dev_pub_genre1 = set(
        df_genre[df_genre['genre'] == genre1]['developer_publisher']
    )
    dev_pub_genre2 = set(
        df_genre[df_genre['genre'] == genre2]['developer_publisher']
    )
    shared = len(dev_pub_genre1.intersection(dev_pub_genre2))
    if shared > 0:
        shared_dev_pub[(genre1, genre2)] = shared

In [5]:
df_genre_weight = (
    pd.DataFrame(
        [(genre1, genre2, weight)
         for (genre1, genre2), weight in shared_dev_pub.items()],
        columns=['genre1', 'genre2', 'weight']
    )
)

df_genre_weight

,genre1,genre2,weight
0,Tutorial,Documentary,2
1,Tutorial,Indie,2
2,Tutorial,Simulation,2
3,Tutorial,Accounting,2
4,Tutorial,Software Training,2
...,...,...,...
326,Education,Audio Production,5
327,Education,Design & Illustration,27
328,Casual,Audio Production,8
329,Casual,Design & Illustration,19
